<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/main/Correction%2BPrepareDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAVORO PER SARDE:
1. Aggiungere altri todo
2. Imparare ad essere umile
3. Tifare il Campobasso

TODO: CONTROLLARE I PARAGRAFI

1. Controllare se non ci sono whitespace dopo gli .\n Maiuscola
2. Score value spacy per identificare la similarità, potremmo usarla per paragrafi diversi.

TODO: CONTROLLARE I NOMI PROPRI
1. Si potrebbe effettuare il lower() prima di aggiungere le parole allo spellchecker sperando che ci siano i nomi propri necessari alla stesura del riassunto all'interno del prompt
2. Controllare i nomi propri facendo ner e non tenerli in considerazione per eventuali errori, ma confrontarli con eventuali nomi propri all'interno dei prompt (selezionare i soggetti che iniziano per le stesse lettere e poi calcolarcisi l'edit distance ==> wording, entro un certo range viene considerato match e si salva una feature per il numero di match con i prompt ==> content)



In [89]:
!pip install pyspellchecker sympy spacy autocorrect

In [90]:
!git clone "https://github.com/Sopralapanca/CommonLitChallenge.git"

fatal: destination path 'CommonLitChallenge' already exists and is not an empty directory.


In [91]:
import pandas as pd
summaries_train_path = "/content/CommonLitChallenge/data/summaries_train.csv"
prompt_train_path = "/content/CommonLitChallenge/data/prompts_train.csv"

train_data = pd.read_csv(summaries_train_path, sep=',', index_col=0)
prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)

In [92]:
prompt_data.head()

,prompt_question,prompt_title,prompt_text
prompt_id,,,
39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \nAs the sequel to what has already...
3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [93]:
import itertools
word_list = []
for prompt, query, title in zip(prompt_data.prompt_text.tolist(), prompt_data.prompt_question.tolist(), prompt_data.prompt_title.tolist()):
  word_list.append(prompt.replace('\n', '').split())
  word_list.append(query.replace('\n', '').split())
  word_list.append(title.replace('\n', '').split())
token_list = list(itertools.chain(*word_list))

In [94]:
from autocorrect import Speller

class SpellCorrector():
  def __init__(self) -> None:
    self.speller = Speller(lang='en')
  def initialize_tokens(self, tokens):
    self.speller.nlp_data.update({token:1000 for token in tokens})
  def run(self, dataframe):
    dataframe["fixed_summary_text"] = dataframe["text"].apply(
            lambda x: self.speller(x)
        )
    return dataframe["fixed_summary_text"]

In [95]:
spell_enchanter = SpellCorrector()
spell_enchanter.initialize_tokens(token_list)
train_data["fixed_summary_text"] = spell_enchanter.run(train_data)

In [96]:
from nltk.util import ngrams
from collections import Counter
import nltk
nltk.download('punkt')  # Download the required resources for tokenization

def count_ngrams(text, n):
  words = nltk.word_tokenize(text)
  ngram_counts = Counter(ngrams(words, n))
  return ngram_counts

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [97]:
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocessText(text, removal=True):
    # replace newline with space
    text = text.replace("\n", " ")
    text = text.replace('\r', ' ')

    # Normalize spaces around punctuation marks
    text = re.sub(r"[^A-Za-z0-9']", r' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Replace curly apostrophe with straight single quote
    text = text.replace('’', "'")

    # Normalize spaces around punctuation marks
    text = text.strip()

    # lower case
    text = text.lower()

    if removal:
      # split text
      words = text.split()

      # stop word removal
      words = [w for w in words if not w in stop_words]

      # stemming
      # words = [stemmer.stem(w) for w in words]

      # lemmatization
      words = [lemmatizer.lemmatize(w) for w in words]

      text = ' '.join(words)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [98]:
del_columns = []

for n in range(2,5):
  col = f"{n}grams-prompttext-count"
  prompt_data[col] = prompt_data.apply(lambda row: count_ngrams(row["prompt_text"], n), axis=1)

  del_columns.append(col)

  col = f"{n}grams-correctedtext-count"
  train_data[col] = train_data.apply(lambda row: count_ngrams(row["fixed_summary_text"], n), axis=1)

  del_columns.append(col)

  col = f"{n}grams-text-count"
  train_data[col] = train_data.apply(lambda row: count_ngrams(row["text"], n), axis=1)

  del_columns.append(col)

prompt_data["prompt_text_length"] = prompt_data["prompt_text"].apply(len)
prompt_data["prompt_text_preprocessed"] = prompt_data["prompt_text"].apply(lambda x: preprocessText(x))

In [99]:
training_data = train_data.merge(prompt_data, on='prompt_id')

In [100]:
def count_cooccurring_ngrams(text, prompt_text):
    cooccurring_count = sum((text & prompt_text).values())
    return cooccurring_count

In [101]:
for n in range(2,5):

  text_col = f"{n}grams-text-count"
  corre_col = f"{n}grams-correctedtext-count"
  prompt_col = f"{n}grams-prompttext-count"

  new_col=f"{n}grams-correct-count"
  training_data[new_col] = training_data.apply(lambda row: count_cooccurring_ngrams(row[text_col], row[corre_col]), axis=1)

  new_col=f"{n}grams-cooccurrence-count"
  training_data[new_col] = training_data.apply(lambda row: count_cooccurring_ngrams(row[corre_col], row[prompt_col]), axis=1)

In [102]:
training_data.drop(columns=del_columns, inplace=True)

In [103]:
training_data["text_length"] = training_data["fixed_summary_text"].apply(len)
training_data["length_ratio"] = training_data["text_length"] / training_data["prompt_text_length"]

In [104]:
# normalize the data taking into consideration the prompt title

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def normalize_col(training_data, col):
  # Create a new DataFrame to store the normalized text length values
  normalized_df = pd.DataFrame()

  # Group by 'prompt_title' and apply the normalization separately for each group
  for title, group in training_data.groupby('prompt_title'):
      normalized_text_length = scaler.fit_transform(group[[col]])
      new_name = "normalized_"+col
      group[new_name] = normalized_text_length
      normalized_df = pd.concat([normalized_df, group])
  training_data = normalized_df.copy()
  return training_data

In [106]:
!python -m spacy download en_core_web_sm

2023-08-25 11:32:17.079619: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [107]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [108]:
def semantic_similarity(text1, text2, preprocess=False):
  if preprocess:
    text1 = preprocessText(text1)
    text2 = preprocessText(text2)

  embedding1 = nlp(text1)
  embedding2 = nlp(text2)

  return embedding1.similarity(embedding2)

In [109]:
training_data["text_preprocessed"] = training_data["text"].apply(lambda x: preprocessText(x, False))
training_data["correctedtext_preprocessed"] = training_data["fixed_summary_text"].apply(lambda x: preprocessText(x, False))
training_data["correctedtext_pre_no_stop"] = training_data["fixed_summary_text"].apply(lambda x: preprocessText(x))
training_data["semantic_similarity"] = training_data.apply(lambda row: semantic_similarity(row["correctedtext_pre_no_stop"], row["prompt_text_preprocessed"]), axis=1)

<ipython-input-108-34d691352d0d>:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  return embedding1.similarity(embedding2)


In [110]:
from spellchecker import SpellChecker

spell = SpellChecker()

spell.word_frequency.load_words(token_list)

def misspelled_counter(text):
  words = text.split()
  misspelled = spell.unknown(words)
  return len(misspelled)

def misspelled(text):
  words = text.split()
  misspelled = spell.unknown(words)
  return misspelled

In [111]:
training_data["text_misspelled_counter"] = training_data["text_preprocessed"].apply(lambda x: misspelled_counter(x))
training_data["corrected_misspelled_counter"] = training_data["correctedtext_preprocessed"].apply(lambda x: misspelled_counter(x))

training_data["text_misspelled_words"] = training_data["text_preprocessed"].apply(lambda x: misspelled(x))
training_data["corrected_misspelled_words"] = training_data["correctedtext_preprocessed"].apply(lambda x: misspelled(x))

In [120]:
normalize_cols = ["text_length", "text_misspelled_counter", "corrected_misspelled_counter", "2grams-cooccurrence-count", "2grams-correct-count",
                  "3grams-cooccurrence-count", "3grams-correct-count", "4grams-cooccurrence-count", "4grams-correct-count"]
for col in normalize_cols:
  training_data = normalize_col(training_data, col)
normalize_cols.extend(['prompt_text_length'])
training_data.drop(columns=normalize_cols, inplace=True)

In [121]:
def add_row(df1, df2, preprocess=False):
  row = df2.unique().tolist()[0]
  if preprocess:
    row = preprocessText(row)
  combined_data = pd.concat([pd.Series([row]),df1.loc[:]]).reset_index(drop=True) #append row on the head of the dataframe
  return combined_data

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Group by 'prompt_id' and compute TF-IDF separately for each class
tfidf_vectorizers = {}

for class_id, group in training_data.groupby('prompt_id'):
    #text_data = group['text'].apply(preprocessText)
    text_data = group['correctedtext_pre_no_stop']

    prompt_question_data = group['prompt_question']
    prompt_title_data = group['prompt_title']
    prompt_text_data = group['prompt_text_preprocessed']

    # Concatenate the preprocessed data for TF-IDF calculation
    combined_data = add_row(text_data, prompt_question_data, True)
    combined_data = add_row(combined_data, prompt_title_data, True)
    combined_data = add_row(combined_data, prompt_text_data)


    # Compute TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data)
    tfidf_vectorizers[class_id] = {'vectorizer': tfidf_vectorizer, 'matrix': tfidf_matrix}

In [123]:
import sympy

In [124]:
karp_tfidf_scores = {}

# Calculate TF-IDF scores for each document
for class_id, group in training_data.groupby('prompt_id'):
    tfidf_vectorizer = tfidf_vectorizers[class_id]['vectorizer']
    tfidf_matrix = tfidf_vectorizers[class_id]['matrix']
    tfidf_matrix = tfidf_matrix[3:] #remove first 3 rows f the matrix since they belongs to prompt_text, prompt_question, prompt_title

    modulus = sympy.randprime(tfidf_matrix.shape[1]-100, tfidf_matrix.shape[1])

    # Iterate through documents and calculate TF-IDF scores
    for index, row in group.iterrows():
        doc_tfidf = tfidf_matrix[index - group.index[0]].toarray()[0]

        doc_tfidf = doc_tfidf[doc_tfidf>0]

        # Calculate the average TF-IDF score for the document
        #average_tfidf_score = sum(doc_tfidf) / len(doc_tfidf)

        #average_tfidf_scores[index] = average_tfidf_score

        gamma = 1e-2
        single_tfidf_score = sum([t**(gamma*i) for i, t in enumerate(doc_tfidf)]) % modulus

        karp_tfidf_scores[index] = single_tfidf_score

In [125]:
# Add the calculated average TF-IDF scores as a new column to the DataFrame
training_data['karp_tfidf_scores'] = [karp_tfidf_scores[index] for index in training_data.index]

In [126]:
normalize_cols = ["karp_tfidf_scores"]
for col in normalize_cols:
  training_data = normalize_col(training_data, col)

In [127]:
training_data.drop("karp_tfidf_scores", axis=1, inplace=True)

In [128]:
training_data.rename(columns={"normalized_karp_tfidf_scores": "karp_tfidf_scores"}, inplace=True)

In [129]:
training_data.head()

,prompt_id,text,content,wording,fixed_summary_text,prompt_question,prompt_title,prompt_text,prompt_text_preprocessed,length_ratio,...,corrected_misspelled_words,normalized_text_misspelled_counter,normalized_corrected_misspelled_counter,normalized_2grams-cooccurrence-count,normalized_2grams-correct-count,normalized_3grams-cooccurrence-count,normalized_3grams-correct-count,normalized_4grams-cooccurrence-count,normalized_4grams-correct-count,karp_tfidf_scores
3099,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In Egypt, there were many occupations and soci...","In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,egyptian society structured like pyramid top g...,0.411235,...,"{siris, peoples'}",0.076923,0.166667,0.138889,0.349381,0.051151,0.355062,0.020725,0.360335,0.956539
3100,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,The highest class was Pharaohs these people we...,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,egyptian society structured like pyramid top g...,0.047161,...,{2nd},0.076923,0.083333,0.015152,0.017882,0.012788,0.020804,0.010363,0.023743,0.156903
3101,3b9047,The Egyptian society is really different from ...,0.205683,0.380538,The Egyptian society is really different from ...,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,egyptian society structured like pyramid top g...,0.135777,...,{},0.076923,0.000000,0.020202,0.089409,0.000000,0.091540,0.000000,0.093575,0.460427
3102,3b9047,We have the gods and then Logan and If Logan ...,-1.547163,-1.461245,We have the gods and then Logan and If Logan ...,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,egyptian society structured like pyramid top g...,0.039652,...,{wh},0.076923,0.083333,0.002525,0.019257,0.000000,0.022191,0.000000,0.025140,0.079536
3103,3b9047,The social classes are different because they ...,-0.066112,-0.715083,The social classes are different because they ...,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,egyptian society structured like pyramid top g...,0.131271,...,{},0.000000,0.000000,0.103535,0.103164,0.089514,0.108183,0.088083,0.113128,0.537596


In [130]:
training_data.corrwith(training_data["content"])

<ipython-input-130-3ecd97e2210c>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["content"])


content                                    1.000000
wording                                    0.751380
length_ratio                               0.777299
normalized_text_length                     0.781834
semantic_similarity                        0.537417
normalized_text_misspelled_counter         0.275465
normalized_corrected_misspelled_counter    0.194798
normalized_2grams-cooccurrence-count       0.538317
normalized_2grams-correct-count            0.778925
normalized_3grams-cooccurrence-count       0.388556
normalized_3grams-correct-count            0.773245
normalized_4grams-cooccurrence-count       0.338966
normalized_4grams-correct-count            0.768883
karp_tfidf_scores                          0.837049
dtype: float64

In [131]:
training_data.corrwith(training_data["wording"])

<ipython-input-131-360ae61d465e>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["wording"])


content                                    0.751380
wording                                    1.000000
length_ratio                               0.546496
normalized_text_length                     0.556227
semantic_similarity                        0.288848
normalized_text_misspelled_counter         0.218678
normalized_corrected_misspelled_counter    0.172347
normalized_2grams-cooccurrence-count       0.206662
normalized_2grams-correct-count            0.552565
normalized_3grams-cooccurrence-count       0.051939
normalized_3grams-correct-count            0.546936
normalized_4grams-cooccurrence-count       0.013728
normalized_4grams-correct-count            0.544816
karp_tfidf_scores                          0.566751
dtype: float64

In [132]:
training_data.to_csv('dataset.csv', index=False)